In [ ]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

In [ ]:
# ================== DATA ==================
# Waktu proses di tiap mesin (Job vs Machine)
processing_times = np.array([
    [10, 15, 20, 12],  # J1
    [12, 18, 25, 10],  # J2
    [14, 11, 22, 17],  # J3
    [13, 16, 18, 14],  # J4
    [11, 13, 21, 19]   # J5
])

In [ ]:
num_jobs = processing_times.shape[0]  # Jumlah pekerjaan
num_machines = processing_times.shape[1]  # Jumlah mesin

In [ ]:
# ================== KONFIGURASI GENETIC ALGORITHM ==================
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Kita ingin meminimalkan makespan
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_permutation", random.sample, range(num_jobs), num_jobs)  # Representasi individu: urutan pekerjaan
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_permutation)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
# ================== FUNGSI FITNESS ==================
def makespan(individual):
    """Menghitung makespan berdasarkan urutan pekerjaan"""
    schedule = np.zeros((num_machines, num_jobs))  # Menyimpan waktu selesai di tiap mesin
    for job_index, job in enumerate(individual):
        for machine in range(num_machines):
            if job_index == 0:
                if machine == 0:
                    schedule[machine][job] = processing_times[job][machine]
                else:
                    schedule[machine][job] = schedule[machine-1][job] + processing_times[job][machine]
            else:
                if machine == 0:
                    schedule[machine][job] = schedule[machine][individual[job_index-1]] + processing_times[job][machine]
                else:
                    schedule[machine][job] = max(schedule[machine-1][job], schedule[machine][individual[job_index-1]]) + processing_times[job][machine]
    
    return max(schedule[-1]),  # Mengembalikan makespan (total waktu penyelesaian)

In [ ]:
toolbox.register("evaluate", makespan)
toolbox.register("mate", tools.cxUniformPartialyMatched)  # Crossover dengan metode Uniform PMX
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)  # Mutasi: swap dua pekerjaan
toolbox.register("select", tools.selTournament, tournsize=3)  # Seleksi: tournament size 3

In [ ]:
# ================== ALGORITMA EVOLUSI ==================
def run_ga():
    """Menjalankan Genetic Algorithm untuk optimasi penjadwalan produksi"""
    pop = toolbox.population(n=50)  # Populasi awal
    hof = tools.HallOfFame(1)  # Simpan solusi terbaik
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)

    # Jalankan algoritma
    algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.2, ngen=100, stats=stats, halloffame=hof, verbose=True)

    return hof[0], hof[0].fitness.values[0]  # Return solusi terbaik

In [ ]:
# ================== EKSEKUSI ==================
best_schedule, best_makespan = run_ga()
print(f"Jadwal terbaik: {best_schedule}")
print(f"Makespan terbaik: {best_makespan}")